In [12]:
import os
import pandas as pd
import subprocess

# generate an empty dataframe to be filled with each sample sequenced in the experiment
ntc_sample_df = pd.DataFrame()
sc_sample_df = pd.DataFrame()

# assign the file path to the experiments sequencing data
directory = "/Volumes/projects_secondary/shen/projects/2022_07_07_RMG2_Data/SHEH_20220214_WGBS_scWGBS/processed_data/scWGBS/analysis/align/"

# iterate over each file in the directory, remove duplicate names, and populate lists with a minimal sample name as well as the cell counts
ntc_file_list = []
ntc_sample_list = []
ntc_cell_count_list = []

sc_file_list = []
sc_sample_list = []
sc_cell_count_list = []

for filename in os.listdir(directory):
    if "sorted.markdup.bam.bai" in filename:
        if "_0" in filename:
            ntc_cell_count_list.append(0)
            ntc_file_list.append(filename[:24])
            ntc_sample_list.append(filename[:5])
        if "_1" in filename:
            sc_cell_count_list.append(1)
            sc_file_list.append(filename[:24])
            sc_sample_list.append(filename[:5])       

# fill the sample data frame in with the list generated from the experiment directory
ntc_sample_df["file"] = ntc_file_list
ntc_sample_df["sample"] = ntc_sample_list
ntc_sample_df["cell_count"] = ntc_cell_count_list

sc_sample_df["file"] = sc_file_list
sc_sample_df["sample"] = sc_sample_list
sc_sample_df["cell_count"] = sc_cell_count_list

# randomly take a subset of the samples sent to sequencing
ntc_test_df = ntc_sample_df.sample(frac=0.4)
sc_test_df = sc_sample_df.sample(frac=0.05)
test_df = pd.concat([ntc_test_df, sc_test_df])

# read the data into permanent storage and optionally check the samples in the test dataframe
# test_df.to_csv("/Users/David.Sokol/Documents/experiments/20220718_scwgbs_contamination_analysis/data/test_df.csv")
test_df #;

,file,sample,cell_count
4,A06_0.sorted.markdup.bam,A06_0,0
6,B06_0.sorted.markdup.bam,B06_0,0
0,D06_0.sorted.markdup.bam,D06_0,0
59,F11_1.sorted.markdup.bam,F11_1,1
56,D01_1.sorted.markdup.bam,D01_1,1
11,F10_1.sorted.markdup.bam,F10_1,1


In [13]:
# terminal command to get chromosome specific reads
# samtools view -c -F 0x4 /file chr1 (chr2, chrX, chrY)
x = 1
while x <= 24:
    if x == 23:
        chr = "chrX"
    elif x == 24:
        chr = "chrY"
    else:
        chr = "chr" + str(x)

    chr_list = []

    for filename in test_df["file"]:
        chr_reads = subprocess.run(["samtools", "view", "-c", "-F", "0x4", directory + filename, chr], stdout=subprocess.PIPE, text=True)
        chr_list.append(chr_reads.stdout[:-1])

    test_df[chr] = chr_list
    x += 1

# optionally check the read counts mapping to each chromosome of each sample in the dataframe 
test_df #;

,file,sample,cell_count,chr1,chr2,chr3,chr4,chr5,chr6,chr7,...,chr15,chr16,chr17,chr18,chr19,chr20,chr21,chr22,chrX,chrY
4,A06_0.sorted.markdup.bam,A06_0,0,788,1945,640,875,1105,477,998,...,275,494,317,229,341,305,196,274,707,79
6,B06_0.sorted.markdup.bam,B06_0,0,196758,350074,219828,144038,171211,119287,188950,...,71478,93397,82751,50048,81870,90936,55717,86933,135932,9549
0,D06_0.sorted.markdup.bam,D06_0,0,495209,713536,448474,334738,343263,240618,320277,...,150592,191267,173084,101996,173011,182343,114567,193443,272739,19480
59,F11_1.sorted.markdup.bam,F11_1,1,219152,346247,220362,165834,171549,118818,190454,...,74473,95499,85291,50365,99103,91513,57702,96567,132343,9359
56,D01_1.sorted.markdup.bam,D01_1,1,282699,511280,398890,272223,245213,171781,227863,...,107063,133457,120433,73151,122761,129033,80876,135745,191780,13955
11,F10_1.sorted.markdup.bam,F10_1,1,218973,375966,245461,183436,190339,132794,176577,...,82958,104311,93206,56210,94104,101409,62239,104331,148289,10949


In [14]:
# terminal command to get total reads
# samtools view -c -F 0x4 /file
chr_list = []

for filename in test_df["file"]:
        chr_reads = subprocess.run(["samtools", "view", "-c", "-F", "0x4", directory + filename], stdout=subprocess.PIPE, text=True)
        chr_list.append(chr_reads.stdout[:-1])
        
test_df["total_reads"] = chr_list

# convert the numberic collumns in the dataframe to integers
test_df["total_reads"] = test_df["total_reads"].astype(int)
test_df["cell_count"] = test_df["cell_count"].astype(int)

# optionally check the read counts mapping to each chromosome of each sample in the dataframe 
test_df #;

,file,sample,cell_count,chr1,chr2,chr3,chr4,chr5,chr6,chr7,...,chr16,chr17,chr18,chr19,chr20,chr21,chr22,chrX,chrY,total_reads
4,A06_0.sorted.markdup.bam,A06_0,0,788,1945,640,875,1105,477,998,...,494,317,229,341,305,196,274,707,79,13862
6,B06_0.sorted.markdup.bam,B06_0,0,196758,350074,219828,144038,171211,119287,188950,...,93397,82751,50048,81870,90936,55717,86933,135932,9549,2985418
0,D06_0.sorted.markdup.bam,D06_0,0,495209,713536,448474,334738,343263,240618,320277,...,191267,173084,101996,173011,182343,114567,193443,272739,19480,6114118
59,F11_1.sorted.markdup.bam,F11_1,1,219152,346247,220362,165834,171549,118818,190454,...,95499,85291,50365,99103,91513,57702,96567,132343,9359,3033992
56,D01_1.sorted.markdup.bam,D01_1,1,282699,511280,398890,272223,245213,171781,227863,...,133457,120433,73151,122761,129033,80876,135745,191780,13955,4374589
11,F10_1.sorted.markdup.bam,F10_1,1,218973,375966,245461,183436,190339,132794,176577,...,104311,93206,56210,94104,101409,62239,104331,148289,10949,3276243


In [18]:
# create a collumn corresponding to the percentage of reads mapping to each chromosome
total_read_list = [13862, 2985418, 6114118, 3033992, 4374589, 3276243] # FIXME: get rid of the manual imput of total reads for each sample

x = 1
while x <= 24:
    if x == 23:
        chr = "chrX"
    elif x == 24:
        chr = "chrY"
    else:
        chr = "chr" + str(x)

    # convert the numeric collumns to integers
    test_df[chr] = test_df[chr].astype(int)

    chr_list = []
    y = 0 # references back to the index of the total read list

    for count in test_df[chr]:
        chr_percent = (count / total_read_list[y]) * 100
        chr_list.append(chr_percent)
        y += 1
    
    test_df[chr + "_percent"] = chr_list
    x += 1

# optionally check the percentage of total reads mapping to each chromosome of each sample in the dataframe 
test_df #;   


,file,sample,cell_count,chr1,chr2,chr3,chr4,chr5,chr6,chr7,...,chr15_percent,chr16_percent,chr17_percent,chr18_percent,chr19_percent,chr20_percent,chr21_percent,chr22_percent,chrX_percent,chrY_percent
4,A06_0.sorted.markdup.bam,A06_0,0,788,1945,640,875,1105,477,998,...,1.983841,3.563699,2.286827,1.651998,2.459962,2.200260,1.413937,1.976627,5.100274,0.569903
6,B06_0.sorted.markdup.bam,B06_0,0,196758,350074,219828,144038,171211,119287,188950,...,2.394238,3.128440,2.771840,1.676415,2.742330,3.046006,1.866305,2.911921,4.553198,0.319855
0,D06_0.sorted.markdup.bam,D06_0,0,495209,713536,448474,334738,343263,240618,320277,...,2.463021,3.128284,2.830891,1.668205,2.829697,2.982327,1.873811,3.163874,4.460807,0.318607
59,F11_1.sorted.markdup.bam,F11_1,1,219152,346247,220362,165834,171549,118818,190454,...,2.454621,3.147635,2.811181,1.660024,3.266423,3.016257,1.901851,3.182836,4.362009,0.308471
56,D01_1.sorted.markdup.bam,D01_1,1,282699,511280,398890,272223,245213,171781,227863,...,2.447384,3.050732,2.753013,1.672180,2.806229,2.949603,1.848768,3.103034,4.383955,0.319001
11,F10_1.sorted.markdup.bam,F10_1,1,218973,375966,245461,183436,190339,132794,176577,...,2.532108,3.183860,2.844905,1.715685,2.872314,3.095283,1.899706,3.184471,4.526191,0.334194


In [19]:
# use this code snippet to rearrange the data
# opened the full output with a text editor and saved file
x = 1
while x <= 24:
    if x == 23:
        chr = "chrX_percent"
    elif x == 24:
        chr = "chrY_percent"
    else:
        chr = "chr" + str(x) + "_percent"

    for index in test_df.index:
        if len(chr) == 12:
            print((test_df["sample"][index]), test_df["cell_count"][index], chr[3:4], test_df[chr][index])
        elif len(chr) == 13:
            print((test_df["sample"][index]), test_df["cell_count"][index], chr[3:5], test_df[chr][index])
    
    x+=1

# saved to sc_chr_mapping_df.csv

A06_0 0 1 5.684605396046746
B06_0 0 1 6.590634879269838
D06_0 0 1 8.099434783561586
F11_1 1 1 7.223222737568194
D01_1 1 1 6.462298515357672
F10_1 1 1 6.683661743039207
A06_0 0 2 14.031164334150917
B06_0 0 2 11.726130143249621
D06_0 0 2 11.670301423688583
F11_1 1 2 11.412258173390041
D01_1 1 2 11.687497956950928
F10_1 1 2 11.47552242004027
A06_0 0 3 4.616938392728322
B06_0 0 3 7.36339098913452
D06_0 0 3 7.335056340096806
F11_1 1 3 7.263104187486322
D01_1 1 3 9.118342317415419
F10_1 1 3 7.4921487813938095
A06_0 0 4 6.312220458808253
B06_0 0 4 4.824718012687001
D06_0 0 4 5.474837090157567
F11_1 1 4 5.465868070845276
D01_1 1 4 6.222824589921476
F10_1 1 4 5.598974190864353
A06_0 0 5 7.971432693694994
B06_0 0 5 5.73490881343919
D06_0 0 5 5.6142684848411495
F11_1 1 5 5.6542337619875065
D01_1 1 5 5.6053951582651536
F10_1 1 5 5.809672847832105
A06_0 0 6 3.4410618958303276
B06_0 0 6 3.9956548798191744
D06_0 0 6 3.9354490704955314
F11_1 1 6 3.9162265424562754
D01_1 1 6 3.9267917511793677
F10_1 1 

In [20]:
# read the data into permanent storage
test_df.to_csv("/Users/David.Sokol/Documents/experiments/20220718_scwgbs_contamination_analysis/data/edited_test_df.csv")